In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing the necessary libraries**

In [2]:
!pip install matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
plt.style.use("fivethirtyeight")

1. Loading the datasets

In [ ]:
train=pd.read_csv('train.csv')
#test=pd.read_csv('/kaggle/input/santander-pr/test.csv')

In [ ]:
train.shape

**Random selcetion of approx 10% of the samples**

In [ ]:
train_01 = train.sample(n = 1000000)

In [ ]:
import gc#to decrease RAM
del(train)
#train
gc.collect()

In [ ]:
train_01.shape

In [ ]:
train_01.isnull().sum()

In [ ]:
train_01.info()

**Printinf the unique values of all the columns**

In [ ]:
for col in train_01:
    print("_________________________________________________________")
    print(col)
    print(train_01[col].dtype)
    print(train_01[col].unique())


**Data cleaning**

**Conversion of the types to reduce the memory and replacement of unrelated info using np.nan**

In [ ]:
train_01['age'].replace(' NA',np.nan,inplace=True)

In [ ]:
train_01['age'] = train_01['age'].astype('float16')

In [ ]:
train_01['age'] = pd.to_numeric(train_01['age'], errors='coerce')

In [ ]:
train_01['antiguedad'].replace('     NA',np.nan,inplace=True)
train_01['antiguedad'].replace('-999999',np.nan,inplace=True)
train_01['antiguedad'].replace(-999999,np.nan,inplace=True)
train_01.antiguedad = pd.to_numeric(train_01.antiguedad, errors='coerce')
train_01.antiguedad = train_01.antiguedad.astype('float16')

In [ ]:
train_01.indrel_1mes.replace('P',5.0, inplace = True)
train_01.indrel_1mes = pd.to_numeric(train_01.indrel_1mes, errors='coerce')
train_01.indrel_1mes = train_01.indrel_1mes.astype('float16')

In [ ]:
train_01.indrel = train_01.indrel.astype('float16')

In [ ]:
train_01.tipodom = train_01.tipodom.astype('float16')
train_01.cod_prov = train_01.cod_prov.astype('float16')
train_01.ind_actividad_cliente = train_01.ind_actividad_cliente.astype('float16')
train_01.ind_nuevo = train_01.ind_nuevo.astype('float16')

In [ ]:
train_01.segmento.replace('02 - PARTICULARES',2.0,inplace=True)
train_01.segmento.replace('03 - UNIVERSITARIO',3.0,inplace=True)
train_01.segmento.replace('01 - TOP',1.0,inplace=True)
train_01.segmento = pd.to_numeric(train_01.segmento, errors='coerce')
train_01.segmento = train_01.segmento.astype('float16')


In [ ]:
train_01

In [ ]:
train_01.iloc[:,24:]

In [ ]:
gc.collect()

**Changing the dtypes of the columns to decrease memory usage**

In [ ]:
for col in train_01.iloc[:,24:]:
    
    train_01[col] = pd.to_numeric(train_01[col], errors='coerce')
    
    if train_01[col].isnull().sum() > 0:
        train_01[col] = train_01[col].astype('float16')
    
    else:     
        train_01[col] = train_01[col].astype('int8', errors = 'ignore')


train_01.iloc[:,24:].info()

In [ ]:
train_01.head()

In [ ]:
for col in train_01:
    print("_________________________________________________________")
    print(col)
    print(train_01[col].dtype)
    print(train_01[col].unique())


In [ ]:
train_01.columns

In [ ]:
x = train_01.set_index(['ncodpers','fecha_dato' ])
x.sort_index(inplace= True)
x.head(20)


In [ ]:
x.describe().style.background_gradient(cmap='Reds')

**Checking the top 10 provinces and the channels with most number of customers**

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.title('Top 10 channels used by the customers to join')
sns.countplot('canal_entrada',data=train_01,order=pd.value_counts(train_01['canal_entrada']).iloc[:10].index);
plt.subplot(2,2,2)
plt.title('Top 10 provinces with most number of customers')
r=sns.countplot('nomprov',data=train_01,order=pd.value_counts(train_01['nomprov']).iloc[:10].index)
r.set_xticklabels(r.get_xticklabels(),rotation=90);

**Number of customers based on different categorical data**

In [ ]:
fig = plt.figure(figsize=(20,12))

cols=['ind_empleado','sexo','tiprel_1mes','indresi','indext','indfall']
plt.subplot(231)
plt.title('Type of employee', fontsize=14)
sns.countplot(cols[0],data=train_01)

plt.subplot(2,3,2)
plt.title('Male vs Female customer', fontsize=14)
sns.countplot(cols[1],data=train_01)

plt.subplot(233)
plt.title('Relationship of the customer at the beginning of the month', fontsize=14)
sns.countplot(cols[2],data=train_01)

plt.subplot(2,3,4)
plt.title('Residence of the customer in the country or not', fontsize=14)
sns.countplot(cols[3],data=train_01)

plt.subplot(2,3,5)
plt.title('Birth country same as bank country', fontsize=14)
sns.countplot(cols[4],data=train_01)

plt.subplot(2,3,6)
plt.title('Alive or Dead', fontsize=14)
sns.countplot(cols[5],data=train_01)

plt.show()

The above plot indicates the following:
1. The unemployed customers are more in number compared to other employed customers
2. The number of male customers are more than female customers
3. There are lot of inactive customers than active customers
4. The residence of the many customers are same as those where the bank is
5. There are a lot of foreign customers (birth country not the same as where the bank is).
6. A majority of the customers are alive

In [ ]:
import plotly.express as px
values=train_01['segmento'].value_counts()
#names=train_01['segmento'].value_counts().index
names=["Individuals",'College_graduates','VIP']
px.pie(train_01,values=values,names=names,title='Type of account holders')

**Distribution of customers across provinces**

In [ ]:
a=train_01.groupby(by = ["nomprov","fecha_dato"]).nunique().ncodpers.reset_index()
a = a[(a.nomprov != "MADRID") & (a.nomprov !=  "BARCELONA") &(a.nomprov != "VALENCIA") &(a.nomprov != "SEVILLA")]

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='fecha_dato',y='ncodpers',data=a,hue='nomprov',palette='Set1');


In [ ]:
fig = px.histogram(a, x="fecha_dato", y="ncodpers", color="nomprov",hover_data=a.columns)
fig.show()

**Renta vs Features**

In [ ]:
fig = px.scatter(train_01, x="nomprov", y="renta", color='segmento')
fig.show()

**Distribution of type of bank account holders on the basis of age for each province**

In [ ]:
fig = px.scatter(train_01, x="nomprov", y="age", color='segmento')
fig.show()

**Slicing data at a gap of four months for periodic data analysis**

In [ ]:
q1 = train_01[train_01.fecha_dato == "2015-04-28"]
q2 = train_01[train_01.fecha_dato == "2015-08-28"]
q3 = train_01[train_01.fecha_dato == "2015-12-28"]
q4 = train_01[train_01.fecha_dato == "2016-04-28"]
q_all = pd.concat([q1,q2,q3,q4])
q_all

In [ ]:
fig = px.histogram(q_all[q_all.nomprov != "MADRID"], x="nomprov", y="ncodpers", color='fecha_dato')
fig.show()

**Distribution of target values across all provinces at a gap of four months**

In [ ]:
for col in q_all.iloc[:,24:]:
    fig = px.histogram(q_all[q_all.nomprov != "MADRID"], x="nomprov", y=col, color='fecha_dato', title = col)
    fig.show()

In [ ]:
x = train_01.groupby(by = "fecha_dato").count().reset_index()

**Visualizing gains/loss of customers for each province at gap of four months**

In [ ]:
tempdf = q_all.groupby(by= ["nomprov","fecha_dato"]).ncodpers.count().reset_index()

In [ ]:
tempdf = q_all.groupby(by= ["nomprov","fecha_dato"]).ncodpers.count().reset_index()
gain_loss = []

for i in range(len(tempdf)):
    if i > 0:
        if tempdf.loc[i, "nomprov"] == tempdf.loc[i-1, "nomprov"]:
            t_gain_loss = (tempdf.loc[i, "ncodpers"] - tempdf.loc[i-1, "ncodpers"])/tempdf.loc[i-1, "ncodpers"]
            gain_loss.append(t_gain_loss)
            
        else:
            gain_loss.append(0)
    else:
        gain_loss.append(0)
tempdf["gain_loss"] = gain_loss
plt.figure(figsize= (20,30))
sns.barplot(data =tempdf, y="nomprov", x="gain_loss", hue='fecha_dato');

**Visualising gain/loss of target features across provinces at gap of four months **

In [ ]:
tempdf = q_all.groupby(by= ["nomprov","fecha_dato"]).sum().reset_index()

for col in tempdf[['ind_ahor_fin_ult1', 'ind_aval_fin_ult1',
       'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
       'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1',
       'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
       'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1',
       'ind_recibo_ult1']]:
    gain_loss = []

    for i in range(len(tempdf)):
        if i > 0:
            if tempdf.loc[i, "nomprov"] == tempdf.loc[i-1, "nomprov"]:
                t_gain_loss = (tempdf.loc[i, col] - tempdf.loc[i-1, col])
                gain_loss.append(t_gain_loss)

            else:
                gain_loss.append(0)
        else:
            gain_loss.append(0)
    tempdf["gain_loss_"+col] = gain_loss
    plt.figure(figsize= (20,30))
    plt.title("gain_loss_"+col)
    sns.barplot(data =tempdf, y="nomprov", x="gain_loss_"+col, hue='fecha_dato');

**Visualizing employment index with gross household income**

In [ ]:
q_all.groupby(by='ind_empleado')['renta'].max()

In [ ]:

fig = px.box(q_all, color="fecha_dato", y="renta", x ="ind_empleado")
fig.show()